In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
X = df.iloc[:,:-1 ].values
y = df.iloc[:,-1].values

In [8]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
scalar = StandardScaler()

In [11]:
X = scalar.fit_transform(X)

In [12]:
X.shape

(768, 8)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=1)

In [15]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [16]:
model = Sequential()

model.add(Dense(32, input_dim=8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

d:\Programming lang\PYTHON\Deep Learning Notebooks\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.fit(Xtrain, ytrain, epochs=100, batch_size=32, validation_data=(Xtest, ytest))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.3529 - loss: 0.7976 - val_accuracy: 0.3571 - val_loss: 0.7483
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3980 - loss: 0.7361 - val_accuracy: 0.5714 - val_loss: 0.6884
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6048 - loss: 0.6764 - val_accuracy: 0.6753 - val_loss: 0.6441
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6781 - loss: 0.6434 - val_accuracy: 0.7338 - val_loss: 0.6082
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7156 - loss: 0.6115 - val_accuracy: 0.7662 - val_loss: 0.5764
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7115 - loss: 0.5863 - val_accuracy: 0.7662 - val_loss: 0.5527
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7522 - loss: 0.5551 - val_accuracy: 0.7662 - val_loss: 0.5321
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7428 - loss: 0.5367 - val_accuracy: 0.7662 - 

- The above process of hyperparameter tuning is a bit manual and time-consuming. We can use keras-tuner to automate this process.

In [18]:
# 1. How to select appropriate optimizers
# 2. No of nodes in a layer
# 3. No of hidden layers

In [19]:
import keras_tuner as kt

In [20]:
# Selecting Optimizer

def build_model(hp):
    model = Sequential()
    
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer', 
                                      values = ['adam', 'sgd', 'rmsprop', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])       # build 4 models with different optimizers
    
    return model

In [21]:
# Making tuner object

tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy', 
                        max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [22]:
tuner.search(Xtrain, ytrain, epochs=5, validation_data=(Xtest, ytest))

In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [24]:
model = tuner.get_best_models(num_models=1)[0]

d:\Programming lang\PYTHON\Deep Learning Notebooks\venv\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.fit(Xtrain, ytrain, batch_size=32, epochs=100, validation_data=(Xtest, ytest))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7140 - loss: 0.5888 - val_accuracy: 0.7857 - val_loss: 0.5398
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7277 - loss: 0.5592 - val_accuracy: 0.7922 - val_loss: 0.5166
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7420 - loss: 0.5218 - val_accuracy: 0.7922 - val_loss: 0.5002
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7494 - loss: 0.5054 - val_accuracy: 0.8117 - val_loss: 0.4888
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7481 - loss: 0.4900 - val_accuracy: 0.7987 - val_loss: 0.4808
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7709 - loss: 0.4887 - val_accuracy: 0.7922 - val_loss: 0.4764
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7771 - loss: 0.4648 - val_accuracy: 0.7922 - val_loss: 0.4712
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7785 - loss: 0.4674 - val_accuracy: 0.7922 - 

In [27]:
# Number of neurons in a layer

def build_model(hp):
    model = Sequential()
    
    model.add(Dense(hp.Int('units', min_value=8, max_value=128, step=32), activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [28]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy', 
                        max_trials=5,
                        directory='output')

Reloading Tuner from output\untitled_project\tuner0.json


In [29]:
tuner.search(Xtrain, ytrain, epochs=5, validation_data=(Xtest, ytest))

In [30]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [31]:
model = tuner.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 721 (2.82 KB)

 Trainable params: 721 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.fit(Xtrain, ytrain, epochs=100, validation_data=(Xtest, ytest))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7434 - loss: 0.5062 - val_accuracy: 0.7987 - val_loss: 0.4810
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7426 - loss: 0.4899 - val_accuracy: 0.8117 - val_loss: 0.4736
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7669 - loss: 0.4798 - val_accuracy: 0.7987 - val_loss: 0.4689
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7678 - loss: 0.4593 - val_accuracy: 0.8117 - val_loss: 0.4656
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7615 - loss: 0.4733 - val_accuracy: 0.8117 - val_loss: 0.4623
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7607 - loss: 0.4684 - val_accuracy: 0.8117 - val_loss: 0.4632
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7625 - loss: 0.4795 - val_accuracy: 0.8052 - val_loss: 0.4628
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7887 - loss: 0.4457 - val_accuracy: 0.8052 - v

In [34]:
# 3. No of hidden layers

def build_model(hp):
    model = Sequential()
    
    model.add(Dense(72, activation='relu', input_dim=8))
    
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        model.add(Dense(72, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [35]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy', 
                        max_trials=5,
                        directory='hidden_layers')

In [36]:
tuner.search(Xtrain, ytrain, epochs=5, validation_data=(Xtest, ytest))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 26s


In [50]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

d:\Programming lang\PYTHON\Deep Learning Notebooks\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Programming lang\PYTHON\Deep Learning Notebooks\venv\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,025 (187.60 KB)

 Trainable params: 48,025 (187.60 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.fit(Xtrain, ytrain, epochs=100, validation_data=(Xtest, ytest))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7601 - loss: 0.4704 - val_accuracy: 0.7597 - val_loss: 0.5266
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7985 - loss: 0.4028 - val_accuracy: 0.7792 - val_loss: 0.5006
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8219 - loss: 0.3926 - val_accuracy: 0.7727 - val_loss: 0.4891
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8178 - loss: 0.3964 - val_accuracy: 0.7792 - val_loss: 0.5223
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8130 - loss: 0.3959 - val_accuracy: 0.7727 - val_loss: 0.5537
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8440 - loss: 0.3753 - val_accuracy: 0.7532 - val_loss: 0.5422
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8064 - loss: 0.3858 - val_accuracy: 0.8052 - val_loss: 0.4883
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8485 - loss: 0.3673 - val_accuracy: 0.7922 - 